# Forecasting Sticker Sales



### Name: Talimul Islam Utsha


[Data](https://www.kaggle.com/competitions/playground-series-s5e1/data)


In [12]:
# Forecasting Sticker Sales

# importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam


In [13]:
# importing datset
df_train= pd.read_csv('train.csv')
df_test= pd.read_csv('test.csv')
submission= pd.read_csv('sample_submission.csv')

In [14]:
df_train.head()

,id,date,country,store,product,num_sold
0,0,2010-01-01,Canada,Discount Stickers,Holographic Goose,NaN
1,1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0
2,2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0
3,3,2010-01-01,Canada,Discount Stickers,Kerneler,423.0
4,4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0


In [15]:
df=df_train.copy()

In [18]:
# shape of the dataset
df.shape


(230130, 6)

In [19]:
df.isnull().sum()

id             0
date           0
country        0
store          0
product        0
num_sold    8871
dtype: int64

In [20]:
df.describe()

,id,num_sold
count,230130.000000,221259.000000
mean,115064.500000,752.527382
std,66432.953062,690.165445
min,0.000000,5.000000
25%,57532.250000,219.000000
50%,115064.500000,605.000000
75%,172596.750000,1114.000000
max,230129.000000,5939.000000


In [21]:
df.columns

Index(['id', 'date', 'country', 'store', 'product', 'num_sold'], dtype='object')

In [22]:

# Display unique values of the "Country" column
unique_countries = df_train['country'].unique()
print("Unique values in the 'Country' column:", unique_countries)

Unique values in the 'Country' column: ['Canada' 'Finland' 'Italy' 'Kenya' 'Norway' 'Singapore']


In [23]:

# Display unique values of the "Store" column
unique_stores = df_train['store'].unique()
print("Unique values in the 'Store' column:", unique_stores)

Unique values in the 'Store' column: ['Discount Stickers' 'Stickers for Less' 'Premium Sticker Mart']


In [24]:

# Display unique values of the "Product" column
unique_products = df_train['product'].unique()
print("Unique values in the 'Product' column:", unique_products)

Unique values in the 'Product' column: ['Holographic Goose' 'Kaggle' 'Kaggle Tiers' 'Kerneler'
 'Kerneler Dark Mode']


In [25]:
# filling null values


from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, r2_score

# Assuming you have already loaded your DataFrame as df_train

# Step 1: Prepare the data
# Fill missing values temporarily for model training
df_train['num_sold'].fillna(-1, inplace=True)  # Temporary fill for model training

# Encode categorical variables
label_encoders = {}
for column in ['country', 'store', 'product']:  # Encode relevant categorical columns
    le = LabelEncoder()
    df_train[column] = le.fit_transform(df_train[column].astype(str))
    label_encoders[column] = le

# Step 2: Define features and target
X = df_train[['country', 'store', 'product']]  # Features
y = df_train['num_sold']  # Target

# Step 3: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Train the Random Forest model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Step 5: Predict missing values in the "num_sold" column
missing_indices = df_train[df_train['num_sold'] == -1].index
if len(missing_indices) > 0:
    # Prepare the data for prediction
    X_missing = df_train.loc[missing_indices, ['country', 'store', 'product']]
    
    # Ensure that X_missing is not empty
    if not X_missing.empty:
        predicted_num_sold = model.predict(X_missing)

        # Fill in the missing values
        df_train.loc[missing_indices, 'num_sold'] = predicted_num_sold
    else:
        print("No data available for num_sold prediction.")
else:
    print("No missing 'num_sold' values to impute.")

# Step 6: Evaluate the model
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error for num_sold Prediction:", mae)
print("R² Score for num_sold Prediction:", r2)

Mean Absolute Error for num_sold Prediction: 97.06391370776541
R² Score for num_sold Prediction: 0.943469571608159


In [28]:
df_train.isnull().sum()

id          0
date        0
country     0
store       0
product     0
num_sold    0
dtype: int64

In [29]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score

# Assuming you have already loaded your DataFrame as df_train

# Prepare the data
X = df_train[['country', 'store', 'product']]  # Features
y = df_train['num_sold']  # Target

# Encode categorical variables
label_encoders = {}
for column in ['country', 'store', 'product']:  # Encode relevant categorical columns
    le = LabelEncoder()
    X[column] = le.fit_transform(X[column].astype(str))
    label_encoders[column] = le

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the XGBoost model
model = xgb.XGBRegressor(random_state=42)

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, 
                           scoring='neg_mean_absolute_error', cv=5, n_jobs=-1, verbose=2)

# Fit the model
grid_search.fit(X_train, y_train)

# Get the best model and its parameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Print the best parameters
print("Best Hyperparameters:", best_params)

# Evaluate the best model
y_pred = best_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the results
print("Mean Absolute Error for Best Model:", mae)
print("R² Score for Best Model:", r2)

C:\Users\talim\AppData\Local\Temp\ipykernel_20012\1499309835.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = le.fit_transform(X[column].astype(str))
C:\Users\talim\AppData\Local\Temp\ipykernel_20012\1499309835.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = le.fit_transform(X[column].astype(str))
C:\Users\talim\AppData\Local\Temp\ipykernel_20012\1499309835.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Hyperparameters: {'colsample_bytree': 1.0, 'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 200, 'subsample': 0.8}
Mean Absolute Error for Best Model: 95.96578256566146
R² Score for Best Model: 0.9437197594570963


In [41]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load your test DataFrame (assuming it's named df_test)
# df_test = pd.read_csv('path_to_your_test_data.csv')  # Uncomment and set the correct path

# Prepare the test data
X_test_data = df_test[['country', 'store', 'product']]  # Features

# Encode categorical variables in the test set
for column in ['country', 'store', 'product']:
    if column in label_encoders:
        X_test_data[column] = label_encoders[column].transform(X_test_data[column].astype(str))

# Scale the features
X_test_data = scaler.transform(X_test_data)

# Predict probabilities for each class using the best XGBoost model
predicted_probabilities = best_model.predict(X_test_data)

# Create a submission DataFrame
submission_df = pd.DataFrame({
    'id': df_test['id'],  # Assuming 'id' is in the test DataFrame
    'num_sold': predicted_probabilities.flatten()  # Flatten the array to match the DataFrame structure
})

# Save the submission file with the new name
submission_df.to_csv('submission2.csv', index=False)

print("Submission file created: submission2.csv")

C:\Users\talim\AppData\Local\Temp\ipykernel_20012\2987880365.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_data[column] = label_encoders[column].transform(X_test_data[column].astype(str))
C:\Users\talim\AppData\Local\Temp\ipykernel_20012\2987880365.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_data[column] = label_encoders[column].transform(X_test_data[column].astype(str))
C:\Users\talim\AppData\Local\Temp\ipykernel_20012\2987880365.py:14: SettingWithCopyWarning: 
A value is tryi

Submission file created: submission2.csv
